# Problem 17
Calculate convergence pressure on system at bubble point pressure, and perform flash to standard conditions

In [1]:
import math
from tabulate import tabulate
R = 10.73146 # Universal gas constant, psia-ft3/R-lbm mol

In [2]:
# Build the component list used for this example from Table B-25
names = ['CO2', 'C1', 'C2', 'C3', 'iC4', 'nC4', 'iC5', 'nC5', 'C6', 'F1', 'F2', 'F3']
z = [0.0111, 0.3950, 0.0969, 0.0784, 0.0159, 0.0372, 0.0123, 0.0211, 0.0295, 0.1578, 0.1243, 0.0205]
M = [44.01, 16.04, 30.07, 44.09, 58.12, 58.12, 72.15, 72.15, 86.17, 114.10, 233.10, 455.00]
rho = [31.18, 20.58, 28.06, 31.66, 35.01, 36.45, 39.13, 39.30, 41.19, 47.86, 52.41, 57.72]
pc = [1070.6, 667.8, 707.8, 616.3, 529.1, 550.7, 490.4, 488.6, 436.9, 409.6, 235.1, 134.6]
tc = [547.6, 343.0, 549.8, 665.7, 734.7, 765.3, 828.8, 845.4, 913.4, 1065.5, 1356.0, 1689.1]
w = [0.2310, 0.0115, 0.0908, 0.1454, 0.1756, 0.1928, 0.2273, 0.2510, 0.2957, 0.3255, 0.6538, 1.1489]

ncomps = len(z)
#Print out table to check we've entered everything correctly
print('Table B-25 Component Properties (Problem 17)')
header=['Component', 'z', 'MW', 'Rho', 'Pc', 'Tc', 'w']
table = []
for i in range(ncomps):
    table.append([names[i], z[i], M[i], rho[i], pc[i], tc[i], w[i]])
table.append(['Total:', round(sum(z),5), ' ', ' ', ' ', ' ', ' ', ' '])
print(tabulate(table,headers=header))

Table B-25 Component Properties (Problem 17)
Component         z  MW     Rho    Pc      Tc      w
-----------  ------  -----  -----  ------  ------  ------
CO2          0.0111  44.01  31.18  1070.6  547.6   0.231
C1           0.395   16.04  20.58  667.8   343.0   0.0115
C2           0.0969  30.07  28.06  707.8   549.8   0.0908
C3           0.0784  44.09  31.66  616.3   665.7   0.1454
iC4          0.0159  58.12  35.01  529.1   734.7   0.1756
nC4          0.0372  58.12  36.45  550.7   765.3   0.1928
iC5          0.0123  72.15  39.13  490.4   828.8   0.2273
nC5          0.0211  72.15  39.3   488.6   845.4   0.251
C6           0.0295  86.17  41.19  436.9   913.4   0.2957
F1           0.1578  114.1  47.86  409.6   1065.5  0.3255
F2           0.1243  233.1  52.41  235.1   1356.0  0.6538
F3           0.0205  455.0  57.72  134.6   1689.1  1.1489
Total:       1


In [3]:
# Same modified Wilson function we used in Problem 16
def k_mod_wilson(w, tc, pc, t, p, pk, A2):
    A1 = 1-((p-14.7)/(pk-14.7))**A2 # Eq 3.160 - assuming pressures given in psia, Eq 3.160 is in terms of psig
    tr = t/tc
    pr = p/pc
    return (pc/pk)**(A1-1)*math.exp(5.37*A1*(1+w)*(1-1/tr))/pr # Eq 3.159

In [4]:
#Like we did in Problem 16, except we are solving for pk now instead of pbub
def Fpk(pk, *args):
    w, tc, pc, t, pbub, A2 = args
    ncomps = len(w)
    K = [k_mod_wilson(w[i], tc[i], pc[i], t+460, pbub, pk, A2) for i in range(ncomps)]
    return 1 - sum((z[i]*K[i]) for i in range(ncomps)) # Eq 3.165

In [5]:
pbub = 3100 #psia
t = 236 # deg F
A2 = 0.7 # Defined as A0 in the exercise

from scipy.optimize import brentq
# Calculate convergence pressure pk consistent with specified bubble point pressure and A2
data = (w, tc, pc, t, pbub, A2)
pk = brentq(Fpk,pbub+1,10000, args = data) # Use SciPy Python solver Brentq instead of Newton iterations
K = [k_mod_wilson(w[i], tc[i], pc[i], t+460, pbub, pk, A2) for i in range(ncomps)]

In [6]:
# And print results
print('Convergence pressure pk:', round(pk,1))
print('\nTable B-26 K Values and incipient phas gas composition (Problem 17)')
header=['Component', 'Tri', 'Pri', 'Ki', 'yi']
table = []
for i in range(ncomps):
    table.append([names[i], round((t+460)/tc[i],3), round(pbub/pc[i],3), round(K[i],5), round(z[i]*K[i],4)])
table.append(['Total:', ' ', ' ', ' ', sum(z[i]*K[i] for i in range(ncomps))])
print(tabulate(table,headers=header))


Convergence pressure pk: 4049.7

Table B-26 K Values and incipient phas gas composition (Problem 17)
Component    Tri    Pri     Ki           yi
-----------  -----  ------  -------  ------
CO2          1.271  2.896   1.32415  0.0147
C1           2.029  4.642   1.5378   0.6074
C2           1.266  4.38    1.19631  0.1159
C3           1.046  5.03    0.99072  0.0777
iC4          0.947  5.859   0.86812  0.0138
nC4          0.909  5.629   0.8322   0.031
iC5          0.84   6.321   0.73369  0.009
nC5          0.823  6.345   0.71042  0.015
C6           0.762  7.095   0.6149   0.0181
F1           0.653  7.568   0.46195  0.0729
F2           0.513  13.186  0.18967  0.0236
F3           0.412  23.031  0.04348  0.0009
Total:                               1


In [7]:
# Flash to standard conditions
tsc = 60 #deg F
psc = 14.7 #psia
Ksc = [k_mod_wilson(w[i], tc[i], pc[i], tsc+460, psc, pk, A2) for i in range(ncomps)]

In [8]:
# Rachford Rice equation #4.36
def RR(Fv, *args):
    z, K = args
    return sum(z[i]*(K[i]-1)/(1+Fv*(K[i]-1)) for i in range(len(z)))

data = (z, Ksc)
Fv = brentq(RR,0.0,1.0, data) # Use in-built Python solver brentq

print('Vapor fraction at standard conditions:', round(Fv,6))

Vapor fraction at standard conditions: 0.642665


In [9]:
# Calculate average MW of surface gas
x = [z[i]/(Fv*(Ksc[i]-1)+1) for i in range(ncomps)]
y = [x[i]*Ksc[i] for i in range(ncomps)] # Eq 4.41
mg = [y[i]*M[i] for i in range(ncomps)]
MW_gas_sc = sum(mg)
sg_sc = MW_gas_sc/28.91
print('Separator gas gravity:', round(sg_sc , 3))

Separator gas gravity: 0.946


In [10]:
# Average stock tank oil molecular weight
moi = [x[i]*M[i] for i in range(ncomps)]
voi = [x[i]*M[i]/rho[i] for i in range(ncomps)]
Mo = sum(moi)
Rho_o = Mo / sum(voi)
gamma_o = Rho_o / 62.4
api_o = 141.5/gamma_o - 131.5
Vg = 379 * Fv # scf/mole of feed
Vo = (1-Fv)*Mo/Rho_o/5.614583 #stb/mole of feed
gor = Vg/Vo

In [11]:
# And summarize results
print('Vapor fraction flashed to standard conditions:', round(Fv,6))
print('Seperator gas gravity:', round(sg_sc,6))
print('Stock tank oil gravity:', round(api_o,2), 'deg API')
print('Gas Oil Ratio:', round(gor,1), 'scf/stb')
print('1 mole of feed yields: Gas Volume =', round(Vg,1), 'scf')
print('                       Oil Volume =', round(Vo,4), 'stb')
print(' ')

c = [1/(Ksc[i]-1) for i in range(ncomps)]
mm = [z[i]/(Fv+c[i]) for i in range(ncomps)]

print('Table B-27 Separator Flash Calculation (Problem 17)')
header=['Component', 'Tri', 'Pri', 'Ki', 'zi/(Fg+ci)', 'xi', 'yi', 'moi', 'voi', 'mgi']
table = []
for i in range(ncomps):
    table.append([names[i], round((tsc+460)/tc[i],3), round(psc/pc[i],3), Ksc[i], round(mm[i],4), round(x[i],4), round(y[i],4), round(moi[i],2), round(voi[i],3), round(mg[i],2)])
table.append(['Total:', ' ', ' ', ' ', sum(mm), sum(x), sum(y), round(sum(moi),2), round(sum(voi),3), round(sum(mg),2)])
print(tabulate(table,headers=header))

Vapor fraction flashed to standard conditions: 0.642665
Seperator gas gravity: 0.945552
Stock tank oil gravity: 43.36 deg API
Gas Oil Ratio: 1151.5 scf/stb
1 mole of feed yields: Gas Volume = 243.6 scf
                       Oil Volume = 0.2115 stb
 
Table B-27 Separator Flash Calculation (Problem 17)
Component    Tri    Pri    Ki                        zi/(Fg+ci)      xi      yi     moi    voi    mgi
-----------  -----  -----  ----------------------  ------------  ------  ------  ------  -----  -----
CO2          0.95   0.014  51.27809261792068        0.0168       0.0003  0.0171    0.01  0       0.75
C1           1.516  0.022  288.5957933749961        0.6113       0.0021  0.6134    0.03  0.002   9.84
C2           0.946  0.021  34.41968933035463        0.1441       0.0043  0.1484    0.13  0.005   4.46
C3           0.781  0.024  7.481849931202622        0.0984       0.0152  0.1136    0.67  0.021   5.01
iC4          0.708  0.028  2.655946105943952        0.0128       0.0077  0.0205    0.